# FileSystem (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installing Packages Expecto.......

Installing Packages Expecto........

Installed Packages Expecto, 10.1.0

In [ ]:
#r "nuget:FSharp.Control.AsyncSeq"
#r "nuget:System.Reactive,5.0.0"
#r "nuget:System.Reactive.Linq,6.0.0-preview.1"

Installing Packages FSharp.Control.AsyncSeq System.Reactive System.Reactive.Linq

Installing Packages FSharp.Control.AsyncSeq. System.Reactive. System.Reactive.Linq.

Installing Packages FSharp.Control.AsyncSeq.. System.Reactive.. System.Reactive.Linq..

Installing Packages FSharp.Control.AsyncSeq... System.Reactive... System.Reactive.Linq...

Installing Packages FSharp.Control.AsyncSeq.... System.Reactive.... System.Reactive.Linq....

Installing Packages FSharp.Control.AsyncSeq..... System.Reactive..... System.Reactive.Linq.....

Installing Packages FSharp.Control.AsyncSeq...... System.Reactive...... System.Reactive.Linq......

Installing Packages FSharp.Control.AsyncSeq....... System.Reactive....... System.Reactive.Linq.......

Installing Packages FSharp.Control.AsyncSeq........ System.Reactive........ System.Reactive.Linq........

Installed Packages FSharp.Control.AsyncSeq, 3.2.1 System.Reactive, 5.0.0 System.Reactive.Linq, 6.0.0-preview.1

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Runtime.fs

In [ ]:
open Common

## Operators

In [ ]:
let inline (</>) a b = System.IO.Path.Combine (a, b)

## createTempDirectoryName

In [ ]:
let createTempDirectoryName () =
    let root =
        match System.Reflection.Assembly.GetEntryAssembly().GetName().Name with
        | assemblyName -> assemblyName

    System.IO.Path.GetTempPath ()
    </> $"!{root}"
    </> string (newGuidFromDateTime System.DateTime.Now)

In [ ]:
//// test

createTempDirectoryName ()
|> _contains System.IO.Path.DirectorySeparatorChar

C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-1828-2816-2bab4f2193cd


## createTempDirectory

In [ ]:
let createTempDirectory () =
    let tempFolder = createTempDirectoryName ()
    let result = System.IO.Directory.CreateDirectory tempFolder

    let getLocals () =
        $"tempFolder: {tempFolder} / result: {({|
            Exists = result.Exists
            CreationTime = result.CreationTime
        |})} {getLocals ()}"

    trace Debug (fun () -> "createTempDirectory") getLocals

    tempFolder

In [ ]:
//// test

let tempDirectory = createTempDirectory ()

Directory.Exists tempDirectory
|> _equal true

16:15:18 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-1840-4095-4f31a7b1057b / result: { CreationTime = 2023-07-21 4:15:18 PM
  Exists = true }
True


## waitForFileAccess

In [ ]:
let waitForFileAccess path =
    let rec loop retry = async {
        try
            use _ = new System.IO.FileStream (
                path,
                System.IO.FileMode.Open,
                System.IO.FileAccess.ReadWrite,
                System.IO.FileShare.None
            )
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / message: {ex.Message} / {getLocals ()}"
                trace Warn (fun () -> "waitForFileAccess") getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use stream = new System.IO.FileStream (path, System.IO.FileMode.Open, System.IO.FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
    stream.Seek (0L, SeekOrigin.Begin) |> ignore
    trace Debug (fun () -> "_4") getLocals
    stream.WriteByte (49uy) |> ignore
    trace Debug (fun () -> "_5") getLocals
    stream.Flush ()
    trace Debug (fun () -> "_6") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! _ = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForFileAccess path
    retries |> _isGreaterThan 100
    let text = File.ReadAllText path
    text |> _equal "1"
}
|> Async.runWithTimeout 3000
|> _equal (Some ())

16:15:18 #2 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-1886-8696-88a5af6f7400 / result: { CreationTime = 2023-07-21 4:15:18 PM
  Exists = true }
16:15:18 #3 [Debug] 1
16:15:18 #4 [Debug] 2
16:15:18 #5 [Debug] _1
16:15:18 #6 [Debug] 3
16:15:18 #7 [Debug] _2
16:15:18 #8 [Debug] 4
16:15:18 #9 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-1886-8696-88a5af6f7400\test.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-1886-8696-88a5af6f7400\test.txt' because it is being used by another process.
16:15:20 #10 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-1886-8696-88a5af6f7400\test.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-1886-8696-88a5af6f7400\test.txt' because it is being used by another process.
16:15:20 #11

## deleteDirectoryAsync

In [ ]:
let deleteDirectoryAsync path =
    let rec loop retry = async {
        try
            System.IO.Directory.Delete (path, true)
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / message: {ex.Message} / {getLocals ()}"
                trace Warn (fun () -> "deleteDirectoryAsync") getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test"

let lockDirectory () = async {
    trace Debug (fun () -> "_1") getLocals
    System.IO.File.WriteAllText (path </> "test.txt", "")
    use _ = new System.IO.FileStream (path </> "test.txt", System.IO.FileMode.Open, System.IO.FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    Directory.CreateDirectory path |> ignore
    trace Debug (fun () -> "2") getLocals
    let! _ = lockDirectory () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    do! deleteDirectoryAsync path
    trace Debug (fun () -> "5") getLocals
}
|> Async.runWithTimeout 3000
|> _equal (Some ())

16:15:21 #15 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2121-2110-24211ecd5a60 / result: { CreationTime = 2023-07-21 4:15:21 PM
  Exists = true }
16:15:21 #16 [Debug] 1
16:15:21 #17 [Debug] 2
16:15:21 #18 [Debug] 3
16:15:21 #19 [Debug] _1
16:15:21 #20 [Debug] _2
16:15:21 #21 [Debug] 4
16:15:21 #22 [Warn] deleteDirectoryAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2121-2110-24211ecd5a60\test / message: The process cannot access the file 'test.txt' because it is being used by another process.
16:15:22 #23 [Warn] deleteDirectoryAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2121-2110-24211ecd5a60\test / message: The process cannot access the file 'test.txt' because it is being used by another process.
16:15:23 #24 [Debug] _3
16:15:23 #25 [Debug] 5
FSharpOption<Unit>
      Value: <null>


## deleteFileAsync

In [ ]:
let deleteFileAsync path =
    let rec loop retry = async {
        try
            System.IO.File.Delete path
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / message: {ex.Message} / {getLocals ()}"
                trace Warn (fun () -> "deleteFileAsync") getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new System.IO.FileStream (path, System.IO.FileMode.Open, System.IO.FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! _ = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    do! deleteFileAsync path
    trace Debug (fun () -> "5") getLocals
}
|> Async.runWithTimeout 3000
|> _equal (Some ())

16:15:23 #26 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2349-4961-4a147f5e88ea / result: { CreationTime = 2023-07-21 4:15:23 PM
  Exists = true }
16:15:23 #27 [Debug] 1
16:15:23 #28 [Debug] 2
16:15:23 #29 [Debug] 3
16:15:23 #30 [Debug] _1
16:15:23 #31 [Debug] _2
16:15:23 #32 [Debug] 4
16:15:23 #33 [Warn] deleteFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2349-4961-4a147f5e88ea\test.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2349-4961-4a147f5e88ea\test.txt' because it is being used by another process.
16:15:25 #34 [Warn] deleteFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2349-4961-4a147f5e88ea\test.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2349-4961-4a147f5e88ea\test.txt' because it is being used by another process.
16:15:25

## moveFileAsync

In [ ]:
let moveFileAsync newPath oldPath =
    let rec loop retry = async {
        try
            System.IO.File.Move (oldPath, newPath)
        with ex ->
            if retry % 100 = 0 then
                let getLocals () =
                    $"oldPath: {oldPath} / newPath: {newPath} / message: {ex.Message} / {getLocals ()}"
                trace Warn (fun () -> "moveFileAsync") getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"
let newPath = tempFolder </> "test2.txt"

let lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new System.IO.FileStream (path, System.IO.FileMode.Open, System.IO.FileAccess.ReadWrite)
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! _ = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    do! path |> moveFileAsync newPath
    trace Debug (fun () -> "5") getLocals
    return File.ReadAllText newPath
}
|> Async.runWithTimeout 3000
|> _equal (Some "0")

16:15:25 #37 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2579-7904-7be577650566 / result: { CreationTime = 2023-07-21 4:15:25 PM
  Exists = true }
16:15:25 #38 [Debug] 1
16:15:25 #39 [Debug] 2
16:15:25 #40 [Debug] 3
16:15:25 #41 [Debug] _1
16:15:25 #42 [Debug] _2
16:15:25 #43 [Debug] 4
16:15:25 #44 [Warn] moveFileAsync / oldPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2579-7904-7be577650566\test.txt / newPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2579-7904-7be577650566\test2.txt / message: The process cannot access the file because it is being used by another process.
16:15:27 #45 [Warn] moveFileAsync / oldPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2579-7904-7be577650566\test.txt / newPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2579-7904-7be577650566\test2.txt / message: The process cannot access the file because it is being used by ano

## watchDirectory

In [ ]:
[<RequireQualifiedAccess>]
type FileSystemChangeType =
    | Error
    | Changed
    | Created
    | Deleted
    | Renamed

[<RequireQualifiedAccess>]
type FileSystemChange =
    | Error of exn: exn
    | Changed of path: string * content: string option
    | Created of path: string * content: string option
    | Deleted of path: string
    | Renamed of oldPath: string * (string * string option)


let watchDirectoryWithFilter filter shouldReadContent path =
    let fullPath = System.IO.Path.GetFullPath path
    let getLocals () = $"fullPath: {fullPath} / filter: {filter} / {getLocals ()}"

    let watcher =
        new System.IO.FileSystemWatcher (
            Path = fullPath,
            NotifyFilter = filter,
            EnableRaisingEvents = true,
            IncludeSubdirectories = true
        )

    let getEventPath (path : string) =
        path.Trim().Replace(fullPath, "").TrimStart [| '/'; '\\' |]

    let ticks () =
        System.DateTime.UtcNow.Ticks

    let readContent fullPath =
        if not shouldReadContent
        then None
        else
            try
                waitForFileAccess fullPath |> Async.runWithTimeout 30000 |> ignore
                System.IO.File.ReadAllText fullPath |> Some
            with ex ->
                trace Error (fun () -> $"watchWithFilter / readContent / message: {ex.Message}") getLocals
                None

    let changedStream =
        AsyncSeq.subscribeEvent
            watcher.Changed
            (fun event ->
                ticks (),
                [ FileSystemChange.Changed (getEventPath event.FullPath, readContent event.FullPath) ]
            )

    let deletedStream =
        AsyncSeq.subscribeEvent
            watcher.Deleted
            (fun event ->
                ticks (),
                [ FileSystemChange.Deleted (getEventPath event.FullPath) ]
            )

    let createdStream =
        AsyncSeq.subscribeEvent
            watcher.Created
            (fun event ->
                let path = getEventPath event.FullPath
                let content = readContent event.FullPath
                ticks (), [
                    FileSystemChange.Created (path, content)
                    if Runtime.isWindows () then
                        FileSystemChange.Changed (path, content)
                ])

    let renamedStream =
        AsyncSeq.subscribeEvent
            watcher.Renamed
            (fun event ->
                ticks (), [
                    FileSystemChange.Renamed (
                        getEventPath event.OldFullPath,
                        (getEventPath event.FullPath, readContent event.FullPath)
                    )
                ]
            )

    let errorStream =
        AsyncSeq.subscribeEvent
            watcher.Error
            (fun event -> ticks (), [ FileSystemChange.Error (event.GetException ()) ])

    let stream =
        [
            changedStream
            deletedStream
            createdStream
            renamedStream
            errorStream
        ]
        |> FSharp.Control.AsyncSeq.mergeAll
        |> FSharp.Control.AsyncSeq.map (fun (n, events) ->
            events
            |> List.fold
                (fun (i, events) event ->
                    i + 1L,
                    (n + i, event) :: events)
                (0L, [])
            |> snd
            |> List.rev
        )
        |> FSharp.Control.AsyncSeq.concatSeq

    let disposable =
        newDisposable (fun () ->
            trace Debug (fun () -> "watchWithFilter / Disposing watch stream") getLocals
            watcher.EnableRaisingEvents <- false
            watcher.Dispose ()
        )

    stream, disposable

let watchDirectory path =
    watchDirectoryWithFilter
        (System.IO.NotifyFilters.Attributes
        ||| System.IO.NotifyFilters.CreationTime
        ||| System.IO.NotifyFilters.DirectoryName
        ||| System.IO.NotifyFilters.FileName
        //  ||| System.IO.NotifyFilters.LastAccess
        //  ||| System.IO.NotifyFilters.LastWrite
        ||| System.IO.NotifyFilters.Security
        //  ||| System.IO.NotifyFilters.Size
        )
        path

### testEventsRaw (test)

In [ ]:
//// test

let testEventsRaw
    (watchFn : bool -> string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable)
    write
    =
    let tempDirectory = createTempDirectory ()
    let stream, disposable = watchFn true tempDirectory

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! _ = iter () |> Async.StartChild
        do! Async.Sleep 150
        return! write tempDirectory
    }

    try
        run
        |> Async.runWithTimeout 10000
        |> _equal (Some ())
    finally
        disposable.Dispose ()
        deleteDirectoryAsync tempDirectory |> Async.RunSynchronously

    let eventsLog =
        events
        |> Seq.toList
        |> List.sortBy fst
        |> List.fold
            (fun (prev, acc) (ticks, event) ->
                ticks, (ticks, (if prev = 0L then 0L else ticks - prev), event) :: acc
            )
            (0L, [])
        |> snd
        |> List.rev
        |> List.map (fun (diff, n, event) ->
            let text = $"{n} / {diff} / {event}"
            text.Substring (0, min 100 text.Length)
        )
        |> String.concat "\n"
    let getLocals () = $"eventsLog: \n{eventsLog} / {getLocals ()}"
    trace Debug (fun () -> "testEventsRaw") getLocals

    events
    |> Seq.toList
    |> List.sortBy fst
    |> List.map snd
    |> List.fold
        (fun (map, acc) event ->
            match event with
            | FileSystemChange.Changed (path, (Some _ as content)) ->
                if map |> Map.containsKey path && map.[path] = content
                then map, acc
                else (map |> Map.add path content), event :: acc
            | FileSystemChange.Created (path, _)
            | FileSystemChange.Deleted path ->
                (map |> Map.remove path), event :: acc
            | FileSystemChange.Renamed (oldPath, (newPath, content)) ->
                (map |> Map.remove oldPath |> Map.remove newPath), event :: acc
            | _ -> map, event :: acc
        )
        (Map.empty, [])
    |> snd
    |> List.rev

#### fast (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"a{i}") |> Async.AwaitTask

    do! Async.Sleep 50

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"b{i}") |> Async.AwaitTask

    do! Async.Sleep 50

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt")

    do! Async.Sleep 50

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"c{i}") |> Async.AwaitTask

    do! Async.Sleep 50

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt")

    do! Async.Sleep 50
}

let events = testEventsRaw watchDirectory write

events
|> _sequenceEqual [
    FileSystemChange.Created ("file1.txt", Some "a1")
    FileSystemChange.Changed ("file1.txt", Some "a1")
    FileSystemChange.Created ("file2.txt", Some "a2")
    FileSystemChange.Changed ("file2.txt", Some "a2")

    FileSystemChange.Changed ("file1.txt", Some "b1")
    FileSystemChange.Changed ("file2.txt", Some "b2")

    FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "b1"))
    FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "b2"))

    FileSystemChange.Changed ("file_1.txt", Some "c1")
    FileSystemChange.Changed ("file_2.txt", Some "c2")

    FileSystemChange.Deleted "file_1.txt"
    FileSystemChange.Deleted "file_2.txt"
]

16:15:29 #48 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2900-0090-0974868fc436 / result: { CreationTime = 2023-07-21 4:15:29 PM
  Exists = true }
16:15:29 #49 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2900-0090-0974868fc436\file2.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2900-0090-0974868fc436\file2.txt' because it is being used by another process.
FSharpOption<Unit>
      Value: <null>
16:15:29 #50 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-2900-0090-0974868fc436 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
16:15:29 #51 [Debug] testEventsRaw / eventsLog: 
0 / 638255637292016197 / Created ("file1.txt", Some "a1")
1 / 638255637292016198 / Changed ("file1.txt", Some "a1")
22520 / 638255637292038718 / Create

#### slow (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    let contents =
        [1..n]
        |> List.map (string >> String.replicate 1_000_000)

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{contents.[i - 1]}a") |> Async.AwaitTask

    do! Async.Sleep 100

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{contents.[i - 1]}b") |> Async.AwaitTask

    do! Async.Sleep 100

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt")

    do! Async.Sleep 100

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"{contents.[i - 1]}c") |> Async.AwaitTask

    do! Async.Sleep 100

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt")

    do! Async.Sleep 100
}

let events =
    testEventsRaw watchDirectory write
    |> List.map (function
        | FileSystemChange.Changed (path, Some content) ->
            FileSystemChange.Changed (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
        | FileSystemChange.Created (path, Some content) ->
            FileSystemChange.Created (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
        | FileSystemChange.Renamed (oldPath, (newPath, Some content)) ->
            FileSystemChange.Renamed (
                oldPath,
                (newPath, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
            )
        | event -> event
    )

events
|> _sequenceEqual [
    FileSystemChange.Created ("file1.txt", Some "1a")
    FileSystemChange.Changed ("file1.txt", Some "1a")
    FileSystemChange.Created ("file2.txt", Some "2a")
    FileSystemChange.Changed ("file2.txt", Some "2a")

    FileSystemChange.Changed ("file1.txt", Some "1b")
    FileSystemChange.Changed ("file2.txt", Some "2b")

    FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "1b"))
    FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "2b"))

    FileSystemChange.Changed ("file_1.txt", Some "1c")
    FileSystemChange.Changed ("file_2.txt", Some "2c")

    FileSystemChange.Deleted "file_1.txt"
    FileSystemChange.Deleted "file_2.txt"
]

16:15:30 #52 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3005-0557-024029dda810 / result: { CreationTime = 2023-07-21 4:15:30 PM
  Exists = true }
16:15:30 #53 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3005-0557-024029dda810\file1.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3005-0557-024029dda810\file1.txt' because it is being used by another process.
16:15:30 #54 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3005-0557-024029dda810\file1.txt / message: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3005-0557-024029dda810\file1.txt' because it is being used by another process.
16:15:30 #55 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3005-0557-024029dda810\file2.txt 

### testEventsSorted (test)

In [ ]:
//// test

let sortEvent event =
    match event with
    | FileSystemChange.Error _ -> 0
    | FileSystemChange.Created _ -> 1
    | FileSystemChange.Changed _ -> 2
    | FileSystemChange.Renamed (_oldPath, _) -> 3
    | FileSystemChange.Deleted _ -> 4

let formatEvents events =
    events
    |> Seq.toList
    |> List.sortBy (snd >> sortEvent)
    |> List.choose (fun (ticks, event) ->
        match event with
        | FileSystemChange.Error _ ->
            None
        | FileSystemChange.Changed (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Changed)
        | FileSystemChange.Created (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Created)
        | FileSystemChange.Deleted path ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Deleted)
        | FileSystemChange.Renamed (_oldPath, (path, _)) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Renamed)
    )
    |> List.sortBy (fun (_, path, _) -> path)
    |> List.distinctBy (fun (_, path, event) -> path, event)

let testEventsSorted (watchFn : string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable) write =
    let path = createTempDirectory ()
    let stream, disposable = watchFn path

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! child = iter () |> Async.StartChild
        do! Async.Sleep 150
        do! write path
        do! child
    }

    try
        run
        |> Async.runWithTimeout 500
        |> ignore
    finally
        disposable.Dispose ()
        deleteDirectoryAsync path |> Async.RunSynchronously

    let events = formatEvents events

    let eventMap =
        events
        |> List.map (fun (ticks, path, event) -> path, (event, ticks))
        |> List.groupBy fst
        |> List.map (fun (path, events) ->
            let event, _ticks =
                events
                |> List.map snd
                |> List.sortByDescending snd
                |> List.head

            path, event
        )
        |> Map.ofList

    let eventList =
        events
        |> List.map (fun (_ticks, path, event) -> path, event)

    eventMap, eventList

#### create and delete (test)

In [ ]:
//// test

let write path = async {
    let n = 3

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt")
}

let eventMap, eventList = testEventsSorted (watchDirectory false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file1.txt", nameof FileSystemChangeType.Deleted

    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Deleted

    "file3.txt", nameof FileSystemChangeType.Created
    "file3.txt", nameof FileSystemChangeType.Changed
    "file3.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Deleted
    "file2.txt", nameof FileSystemChangeType.Deleted
    "file3.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

16:15:31 #60 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3170-7091-7fd35e80efcd / result: { CreationTime = 2023-07-21 4:15:31 PM
  Exists = true }
16:15:32 #61 [Debug] runWithTimeout / timeout: 500
16:15:32 #62 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3170-7091-7fd35e80efcd / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
( file3.txt, Created )
( file3.txt, Changed )
( file3.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted
      - Key: file3.txt
        Value: Deleted


#### change (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt")
}

let eventMap, eventList = testEventsSorted (watchDirectory false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file1.txt", nameof FileSystemChangeType.Deleted

    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Deleted
    "file2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

16:15:32 #63 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3243-4395-44144ffc28b2 / result: { CreationTime = 2023-07-21 4:15:32 PM
  Exists = true }
16:15:32 #64 [Debug] runWithTimeout / timeout: 500
16:15:32 #65 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3243-4395-44144ffc28b2 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted


#### rename (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt")

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt")
}

let eventMap, eventList = testEventsSorted (watchDirectory false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed

    "file_1.txt", nameof FileSystemChangeType.Renamed
    "file_1.txt", nameof FileSystemChangeType.Deleted

    "file_2.txt", nameof FileSystemChangeType.Renamed
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Deleted
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

16:15:33 #66 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3319-1938-124514f6dda4 / result: { CreationTime = 2023-07-21 4:15:33 PM
  Exists = true }
16:15:33 #67 [Debug] runWithTimeout / timeout: 500
16:15:33 #68 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3319-1938-124514f6dda4 / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted


#### full (test)

In [ ]:
//// test

let write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "") |> Async.AwaitTask

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt")

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt")
}

let eventMap, eventList = testEventsSorted (watchDirectory false) write

[
    "file1.txt", nameof FileSystemChangeType.Created
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Created
    "file2.txt", nameof FileSystemChangeType.Changed

    "file_1.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Renamed
    "file_1.txt", nameof FileSystemChangeType.Deleted

    "file_2.txt", nameof FileSystemChangeType.Changed
    "file_2.txt", nameof FileSystemChangeType.Renamed
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> _sequenceEqual eventList

[
    "file1.txt", nameof FileSystemChangeType.Changed
    "file2.txt", nameof FileSystemChangeType.Changed
    "file_1.txt", nameof FileSystemChangeType.Deleted
    "file_2.txt", nameof FileSystemChangeType.Deleted
]
|> Map.ofList
|> _sequenceEqual eventMap

16:15:34 #69 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3408-0874-04b6b530451c / result: { CreationTime = 2023-07-21 4:15:34 PM
  Exists = true }
16:15:34 #70 [Debug] runWithTimeout / timeout: 500
16:15:34 #71 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230721-1615-3408-0874-04b6b530451c / filter: FileName, DirectoryName, Attributes, CreationTime, Security
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Changed )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted
